In [19]:
import pickle
from random import shuffle
fp = open("./data_set/all_data.pkl", 'rb')
date_set = pickle.load(fp)
shuffle(date_set)
print(date_set[0: 10])

[('banjori', 'emyzthoodivettewl'), ('rovnix', 'f1nyx3252iqxzc2kc3'), ('benign', 'sharepoint'), ('rovnix', 'bvlxpq4c67cyxslekt'), ('emotet', 'pvoagpevglmdfnqh'), ('banjori', 'bdusorcajanunal'), ('banjori', 'vprqinalcentricem'), ('emotet', 'gibwkbqbtwfakrvx'), ('benign', 'excelgratis'), ('banjori', 'dryxardenslavetusul')]


In [12]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
import sklearn
from sklearn.model_selection import train_test_split
#%%
# date_set[0: 10]
features = [i[1] for i in date_set]   #提取域名
label = [i[0] for i in date_set]      #提取标签
print(features[1], label[1])
valid_chars = {x:idx+1 for idx, x in enumerate(set(''.join(features)))} #构造检索字典
y = [0 if x == 'benign' else 1 for x in label]   #目标值修改为0或1
max_features = len(valid_chars) + 1  #？？？
maxlen = np.max([len(x) for x in features])   #获得输入特征的最大长度

thepiratefilmes2 benign


In [13]:
import numpy as np
def process_features(valid_chars, features, maxlen):
    """将特征替换为 检索字典的值 并根据最长特征长度 构造每个特征 无值填充为0"""
    X = [[valid_chars[y] for y in x] for x in features]
    X = sequence.pad_sequences(X, maxlen=maxlen)
    X = np.array(X)
    return X
X = process_features(valid_chars, features, maxlen)
y = np.array(y)
# print(x_train[0: 2], type(x_train))

In [4]:
import matplotlib.pyplot as plt
import time
def show_train_history(train_history,train, velidation):
    """
    可视化训练过程 对比
    :param train_history:
    :param train:
    :param velidation:
    :return:
    """
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[velidation])
    plt.title("Train History")   #标题
    plt.xlabel('Epoch')    #x轴标题
    plt.ylabel(train)  #y轴标题
    plt.legend(['train', 'test'], loc='upper left')  #图例 左上角
    plt.show()

def build_model(max_features, maxlen):
    """ 定义模型 """
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=128, input_length=maxlen))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model
def run():
    
    model = build_model(max_features, maxlen) 
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)   #训练集测试集切割
    
    start_time = time.time()
    history = model.fit(x_train, y_train, epochs=3, batch_size=128, shuffle=True, validation_split=0.1)   #训练
    end_time = time.time()
    print("花费时间为{}".format(end_time-start_time))
    
    model.save('./model/DGA_predict_LSTM_V1.h5')
    
    score, acc = model.evaluate(x_test, y_test, batch_size=128)
    print('评分:', score)
    print('测试集准确率:', acc)
    
    """可视化训练过程"""
    show_train_history(history, 'acc', 'val_acc')    # 训练集准确率与验证集准确率 折线图
    show_train_history(history, 'loss', 'val_loss')  # 训练集误差率与验证集误差率 折线图
    
    predect_label = model.predict(x_test)
    for (a, b, c) in zip(y_test[0: 100], x_test[0: 100], predect_label[0: 100]):
        print("---原域名为：", b)
        print("预测倾向为", a)
        print("真实倾向为", c)
    

run()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 63, 128)           4992      
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 136,705
Trainable params: 136,705
Non-trainable params: 0
_______________

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

---原域名为： [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0 26 31 28  9 26 31 15 20 26 31 31
  5 30  9 14  4  6 31 30 27  3  7  9  4  3 33]
预测倾向为 1
真实倾向为 [0.9999992]
---原域名为： [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  6  9  2 20  8  3 12  2  7 15 28  9 20 26  6]
预测倾向为 0
真实倾向为 [3.1769276e-05]
---原域名为： [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 15
  2  1 14 14  3 31  2 33 33  9 18  7  8  2  4]
预测倾向为 1
真实倾向为 [0.9998009]
---原域名为： [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  9 13  8  3 20 30 27 28  3 28  3 31 14  3]
预测倾向为 0
真实倾向为 [1.9311905e-05]
---原域名为： [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [14]:
from keras.models import load_model
model = load_model('./model/DGA_predict_LSTM_V2.h5')
score, acc = model.evaluate(X[0: 50000], y[0: 50000], batch_size=128)
print(score, acc)

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 14
 29 24 27  7  6 35 14 24 34  7 17 23 24 13 22] 0
50000/50000 [==============================] - 33s 652us/step
1.8541112198638916 0.67676


In [16]:
a = [(0, 132), (2, 124), (3, 156), (4, 158)]
shuffle(a)
a

[(3, 156), (0, 132), (2, 124), (4, 158)]

In [21]:
if True:
    print(1)

1
